In [ ]:
C = 4; L = 100; var_J = 1/C; var_h = 4.; 
q = 21; N_chains = 500; h,J = Potts_fixed_mean_conn(L, C, var_J, var_h, q);

res = run_potts(Int8.(rand(1:q, (L, N_chains))), h, J, 
        N_steps= 100, 
        N_points = 20, 
        q = q,
        verbose = true);




In [ ]:

C = 4; L = 100; var_J = 10^-2*(1/C); var_h = 8.; mean_h = -4.; h,J = Potts_fixed_mean_conn(L, C, var_J, var_h, q, mean_h = mean_h);

temp = 1.0; q = 21; N_chains = 500; N_steps = 10^4; J = exchange_parameters(J_dbd, 0.1, 5000);

N_start_seq = 100;
#= res0 = run_potts(Int8.(rand(1:q, (L, N_start_seq))), h, J, 
        N_steps= N_steps, 
        N_points = 20,
        temp = temp-0.3,
        q = q,
        verbose = true);

start_seq = res0.step_msa[end];=#

w = compute_weights(dbd_msa,
    22, 0.2)[1]; idx_seqs = sample(collect(1:size(dbd_msa,2)), ProbabilityWeights(w), 
    N_start_seq); start_seq = dbd_msa[:, idx_seqs]; w_seqs = w[idx_seqs];



NN_points = 100; steps = unique([
        trunc(Int,10^y) for y in range(log10(1), log10(N_steps), 
                length=NN_points)]); N_points = length(steps); res_all = [
    ]; hams_all = zeros(N_start_seq, N_points, N_chains); 

@time for n in 1:N_start_seq
    res = run_potts(Int8.(hcat([start_seq[:,n] for _ in 1:N_chains]...)), 
        h, 
        J,  
        q=q,
        temp = temp,
        N_points = NN_points, 
        N_steps = N_steps);
    
    if n%1 == 0
        println("Initial seq $(n)")
    end
    
    push!(res_all, res) 
    hams_all[n,:,:] .= ham_dist(res.step_msa)
end


hams_single = zeros(N_start_seq, N_points); chi_dyn_single = zeros(N_start_seq, N_points);
for i in 1:N_start_seq
    for n in 1:N_points
        hams_single[i,n] = mean(ham_dist(res_all[i].step_msa[1], res_all[i].step_msa[n]))
        chi_dyn_single[i,n] = var(ham_dist(res_all[i].step_msa[1], res_all[i].step_msa[n]))
    end
end
      

close("all")
plt.plot()
for i in 1:N_start_seq
    plt.plot(steps, chi_dyn_single[i,:], alpha = 0.05, color = "grey")
end

plt.plot(steps, mean(chi_dyn_single, dims = 1)[1,:], linewidth = 2.0, color = "black", label = "chi_dyn")

plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("chi_dyn_A")
plt.savefig("../single_wt_chi_dyn.png")


hams_single_all = mean(hams_all,dims = 3)[:,:,1]; mean_hams_single_all = mean(hams_single_all, Weights(w[idx_seqs]), dims = 1)[1,:];

close("all")
plt.plot()
for i in 1:N_start_seq
    plt.plot(steps, hams_single_all[i,:], alpha = 0.2, color = "grey")
end

plt.plot(steps, mean_hams_single_all, linewidth = 2.0, color = "black", label = "<[H_A]>")


plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("[H_A]")
plt.savefig("../all_single_wt_mean_ham_dist.png")
    


hams0 = mean(hams_all,dims = 3)[:,:,1]; first_part = mean(hams0 .^2, Weights(w[idx_seqs]), dims = 1)[1,:];
second_part  = (mean(hams0, Weights(w[idx_seqs]), dims = 1)[1,:]) .^2; chi_back = first_part - second_part;

hams0_tot = mean(hams_all .^2 ,dims = 3)[:,:,1]; first_part_tot = mean(hams0_tot, Weights(w[idx_seqs]), dims = 1)[1,:];
chi_tot = first_part_tot - second_part; 

chi_dyn = mean(chi_dyn_single, Weights(w[idx_seqs]), dims = 1)[1,:];

close("all")

plt.plot(res_all[1].steps, chi_dyn, color = "red", label = "chi_dyn")
plt.plot(res_all[1].steps, chi_back, color = "blue", label = "chi_back")
plt.plot(res_all[1].steps, chi_tot, color = "black", label = "chi_tot")

plt.legend()
plt.xscale("log")
plt.xlabel("MCMC steps")

savefig("../suscept_dbd_weight.png")

sum(abs2, chi_tot - chi_dyn - chi_back)


In [ ]:
C = 40; L = 250; var_J = 10^3*(1/C); var_h = 8.; mean_h = -4.;  q = 10; h,J = Potts_fixed_mean_conn(L, C, var_J, var_h, q);
temp = 1.0; N_chains = 1000; N_steps = 10^4;  #J = J .* (-1);

N_start_seq = 200;
res0 = run_potts(Int8.(rand(1:q, (L, N_start_seq))), h, J, 
        N_steps= 10^6, 
        N_points = 20,
        temp = temp,
        q = q,
        verbose = true);

start_seq = res0.step_msa[end];



NN_points = 100; steps = unique([
        trunc(Int,10^y) for y in range(log10(1), log10(N_steps), 
                length=NN_points)]); N_points = length(steps); res_all = [
    ]; hams_all = zeros(N_start_seq, N_points, N_chains); 

@time for n in 1:N_start_seq
    res = run_potts(Int8.(hcat([start_seq[:,n] for _ in 1:N_chains]...)), 
        h, 
        J,  
        q=q,
        temp = temp,
        N_points = NN_points, 
        N_steps = N_steps);
    
    if n%1 == 0
        println("Initial seq $(n)")
    end
    
    push!(res_all, res) 
    hams_all[n,:,:] .= ham_dist(res.step_msa)
end


hams_single = zeros(N_start_seq, N_points); chi_dyn_single = zeros(N_start_seq, N_points);
for i in 1:N_start_seq
    for n in 1:N_points
        hams_single[i,n] = mean(ham_dist(res_all[i].step_msa[1], res_all[i].step_msa[n]))
        chi_dyn_single[i,n] = var(ham_dist(res_all[i].step_msa[1], res_all[i].step_msa[n]))
    end
end
      

close("all")
plt.plot()
for i in 1:N_start_seq
    plt.plot(steps, chi_dyn_single[i,:], alpha = 0.05, color = "grey")
end

plt.plot(steps, mean(chi_dyn_single, dims = 1)[1,:], linewidth = 2.0, color = "black", label = "chi_dyn")

plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("chi_dyn_A")
plt.savefig("../single_wt_chi_dyn.png")


hams_single_all = mean(hams_all,dims = 3)[:,:,1]; mean_hams_single_all = mean(hams_single_all, dims = 1)[1,:];

close("all")
plt.plot()
for i in 1:N_start_seq
    plt.plot(steps, hams_single_all[i,:], alpha = 0.2, color = "grey")
end

plt.plot(steps, mean_hams_single_all, linewidth = 2.0, color = "black", label = "<[H_A]>")


plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("[H_A]")
plt.savefig("../all_single_wt_mean_ham_dist.png")
    


hams0 = mean(hams_all,dims = 3)[:,:,1]; first_part = mean(hams0 .^2, dims = 1)[1,:];
second_part  = (mean(hams0, dims = 1)[1,:]) .^2; chi_back = first_part - second_part;

hams0_tot = mean(hams_all .^2 ,dims = 3)[:,:,1]; first_part_tot = mean(hams0_tot, dims = 1)[1,:];
chi_tot = first_part_tot - second_part; 

chi_dyn = mean(chi_dyn_single, dims = 1)[1,:];

close("all")

plt.plot(res_all[1].steps, chi_dyn, color = "red", label = "chi_dyn")
plt.plot(res_all[1].steps, chi_back, color = "blue", label = "chi_back")
plt.plot(res_all[1].steps, chi_tot, color = "black", label = "chi_tot")

plt.legend()
plt.xscale("log")
plt.xlabel("MCMC steps")

savefig("../suscept_dbd_weight.png")

sum(abs2, chi_tot - chi_dyn - chi_back)
